## About this notebook:

This notebook extracts information from single cells in the 4i experiment. To customize calculated parameters read documentation of scikit-image regionprops: https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops.

Input:
- data frames for selected wells
- labels
- selected round (anchor of alignment)
- punchmask (mask of excluded regions)
- aligned tiffs

Output:
- data frames with features of nuclei and rings

## Fill in info about the experiment to process

In [1]:
# pathway to a directory with data frames (ex. df)
path_df = r'C:\Users\gases\Desktop\DataForAyra\output_df'

# pathway to segmented cells (ex. im_segmented)
path_labels = r'C:\Users\gases\Desktop\DataForAyra\segmented'

# select anchor round
label_round = 0 # 0-indexed

# pathway to punchmasks (ex. masks)
path_mask = r'C:\Users\gases\Desktop\DataForAyra\masks'

# pathway to aligned tiffs (ex. aligned_tiffs)
path_tiffs = r'C:\Users\gases\Desktop\DataForAyra\aligned_tiffs'

# pathway to save output data frames with info about cells (ex. cell data)
path_save = r'C:\Users\gases\Desktop\DataForAyra\cell_data'

In [2]:
well_list = ['B02','B03','B04','B05','B06','B07','B08','B09','B10','B11']

In [3]:
# settings for properties to calculate
properties = ['label', 'area','centroid','orientation','major_axis_length','minor_axis_length','bbox','mean_intensity']
properties_ring = ['label','area', 'mean_intensity']

## Prepare for processing

In [4]:
import os
import pickle
import pickle5 as pickle
import numpy as np
import pandas as pd
from tifffile import imread
from skimage.measure import regionprops_table
from skimage.segmentation import clear_border
from skimage.draw import polygon
from ring_functions import make_rings

In [5]:
# create directory for saving data frames if needed
try:
    os.mkdir(path_save)
    print('Directory for saving cell data created.')
except:
    print('Directory not created.')

Directory for saving cell data created.


## Process

In [6]:
for selected_well in well_list:
    
    print(f'Processing well {selected_well}.')
    
    # open data frame
    df_name = f'df_{selected_well}.pkl'
    df = pd.read_pickle(os.path.join(path_df,df_name))
    
    ##############################################################
    
    # open labels
    print('Loading labels.')
    labels_path = os.path.join(path_labels,selected_well,f'labels_{selected_well}_round_{str(label_round).zfill(3)}.tif')
    labels = imread(labels_path)
    
    # clear border objects
    labels = clear_border(labels)
    
    ##############################################################
    
    # open punchmask
    print('Loading mask.')
    
    # translate shapes into an image
    mask = np.zeros(labels.shape).astype(int)
    
    try:
        masking = os.path.join(path_mask,f'mask_{selected_well}.pkl')
        sh = pickle.load( open( masking, "rb" ) )

        for reg in sh:
            rr, cc = polygon(reg[:,0],reg[:,1]) 
            mask[rr,cc] = 1
            
        print('Existing mask found.')
        
    except:
        print('Mask not found.')
        
    
    ##############################################################
    
    # open intensity images
    print('Loading intensity images.')
    int_im = []
    for ind,signal in df.iterrows():
        
        im_path = os.path.join(path_tiffs,selected_well,signal.aligned_file_name)
        im = imread(im_path)
        
        int_im.append(im)
        
    int_im.append(mask)    
        
    int_im = np.array(int_im)
    int_im = np.moveaxis(int_im,0,2)

        
    ##############################################################
        
    # calculate properties of nuclei
    print('Quantifying nuclei.')
    nuclei_df = pd.DataFrame(regionprops_table(labels, properties=properties, intensity_image=int_im))
    nuclei_df = nuclei_df.rename(columns={'area': 'nuc_area'})

    # adjust names
    channel_list = ['_'.join([str(x).zfill(2), y]) for x, y in zip(df.nameRound, df.signal.to_list())]
    df['round_channel'] = channel_list

    intensity_names = [f'{x}_nuc_mean' for x in df.round_channel]
    intensity_names.append('nuc_mask')

    # Set the column names of nuclei_df
    nuclei_df.columns = list(nuclei_df.columns[:-len(df) - 1]) + intensity_names
    
    ##############################################################
        
    # generate rings
    print('Generating cytoplasmic rings.')
    rings = make_rings(labels,width=6,gap=1)
    
    # calculate properties of rings
    print('Quantifying cytoplasmic rings.')
    rings_df = pd.DataFrame(regionprops_table(rings, properties=properties_ring, intensity_image=int_im))
    rings_df = rings_df.rename(columns={'area': 'ring_area'})
    
    # adjust names
    intensity_names = [f'{x}_ring_mean' for x in df.round_channel]
    intensity_names.append('ring_mask')
    
    rings_df.columns=list(rings_df.columns[:-len(df)-1])+intensity_names

    ##############################################################
        
    # merging data frames
    cell_data = pd.merge(nuclei_df,rings_df,how='inner',on='label',suffixes=('_nuc', '_ring'))

    #trim out rows without DNA
    for column in cell_data.columns:
        if column.endswith('_DNA_nuc_mean'):
            cell_data = cell_data[cell_data[column] != 0]

    #Calculate protein and DNA totals for Nucleus and Rings
    # Get the columns containing 'nuc_mean' in their names
    nuc_mean_columns = [col for col in cell_data.columns if 'nuc_mean' in col]
    ring_mean_columns = [col for col in cell_data.columns if 'ring_mean' in col]

    # Iterate over each nuc_mean column
    for nuc_mean_col in nuc_mean_columns:
        # Extract the common prefix
        prefix = nuc_mean_col.split('_nuc_mean')[0]

        # Multiply each cell in the nuc_mean column by the 'nuc_area' and store the result in total_protein column
        cell_data[f"{prefix}_total_nuc_protein"] = cell_data[nuc_mean_col] * cell_data['nuc_area']


    for nuc_col, ring_col in zip(nuc_mean_columns, ring_mean_columns):
        # Extract the common prefix
        prefix = nuc_col.split('_nuc_mean')[0]
        suffix = ring_col.split('_ring_mean')[0]

        # Multiply each cell in the ring_mean column by the 'nuc_area' and store the result in total_protein column
        cell_data[f"{prefix}_nuc_cyto_ratio"] = cell_data[nuc_col] / cell_data[ring_col]


    ##############################################################
        
    # add info and save 
    print("Ready to Save")
    cell_data['well'] = selected_well

    cell_data.to_pickle(os.path.join(path_save,f'cell_data_{selected_well}_df.pkl'),protocol=4)
    cell_data.to_csv(os.path.join(path_save,f'cell_data_{selected_well}_df.csv'))
    

Processing well C05.
Loading labels.
Loading mask.
Existing mask found.
Loading intensity images.
Quantifying nuclei.
Generating cytoplasmic rings.
Quantifying cytoplasmic rings.
Ready to Save
Processing well C06.
Loading labels.
Loading mask.
Existing mask found.
Loading intensity images.
Quantifying nuclei.
Generating cytoplasmic rings.
Quantifying cytoplasmic rings.
Ready to Save
Processing well C07.
Loading labels.
Loading mask.
Existing mask found.
Loading intensity images.
Quantifying nuclei.
Generating cytoplasmic rings.
Quantifying cytoplasmic rings.
Ready to Save
Processing well C08.
Loading labels.
Loading mask.
Existing mask found.
Loading intensity images.
Quantifying nuclei.
Generating cytoplasmic rings.
Quantifying cytoplasmic rings.
Ready to Save
Processing well C09.
Loading labels.
Loading mask.
Existing mask found.
Loading intensity images.
Quantifying nuclei.
Generating cytoplasmic rings.
Quantifying cytoplasmic rings.
Ready to Save
Processing well C10.
Loading labels